# Introduction
This notebook is part of the NRTK demonstration suite, demonstrating how perturbations can be applied and their impact measured via MAITE evaluation workflows.

## Layout
This notebook demonstrates how a particular condition (in this case, <b><i>extreme illumination conditions</i></b>), can affect an object detection model, and how that impact can be measured. The overall structure is:

- **Traditional vs. relative mAP:**
    - An overview of the nuances of what we'll be evaluating.
- **Setup:**
    - Notebook initialization, loading the supporting python code. Depending on if this is the first time you've run this notebook, this may take some time.
    - Loading the source image, which will be used throughout the notebook.
- **Image perturbation examples:**
    - The NRTK perturbation is demonstrated on the source image.
- **Baseline detections:**
    - The object detection model is loaded and run on the unperturbed image. These will serve as \"ground truth\" for comparisons against the perturbed images.
 
At this point, we have the fundamental elements of our evaluation: the model, our reference image, and a mechanism for creating the perturbed test images. Next we adapt these elements to be used with the MAITE evaluation workflow:

- **Wrapping the detection model**
- **Wrapping the reference image as a dataset**
- **Wrapping the perturbation as augmentation objects**
- **Wrapping the metrics**

After the evaluation elements have been wrapped, we can run the evaluation:

- **Preparing the augmentations:**
    - We specify the range of perturbation values to evaluate and optionally specify which ones we'd like to visualize.
- **Evaluation of augmented data:**
    - Each augmentation is run through MAITE's evaluation workflow, computing the mean average precision metric relative to the unperturbed detections.
- **Evaluation analysis:**
    - We plot and discuss the mAP@50 metric from each of the perturbed images, as well as per-class and per-area results.

# Evaluation guidance: traditional vs. relative mAP

This notebook will be evaluating the perturbed images using mean average precision (mAP) **relative** to detections from the unperturbed image. Traditional mAP scores the computed detections to ground-truth annotations vetted by an analyst; the mAP metric indicates how well the detector does compared to that analyst and thus measures the detector's "absolute" performance ("absolute" in the sense that the assumption is no detector can do better than the analyst.)

In contrast, in this notebook, we're not concerned with the **absolute** ability of the detector to find objects of interest. Rather, we're interested in how the **perturbations** affect the detector *relative to the unperturbed image*. It's expected that the detector won't find every target in the unperturbed image; instead, we're measuring the **change in the detections** (or classifications) caused by the perturbations.

To support relative mAP, we'll be computing detections on the unperturbed image and using those as our "ground truth" dataset, and using the MAITE dataset class slightly differently than usual. For example, there's no on-disk json file of reference annotations with an associated data loader; instead, we'll be taking the computed detections and manually copying them over into the dataset.

# Setup: Notebook initialization
The next few cells import the python packages used in the rest of the notebook.

**Note:** We are suppressing warnings within this notebook to reduce visual clutter for demonstration purposes. If any issues arise while executing this notebook, we recommend that the first cell is **not** executed so that any related warnings are shown.

In [ ]:
from __future__ import annotations

# warning suppression
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import sys  # noqa: F401

print("Beginning package installation...")
!{sys.executable} -m pip install -qU pip

print("Installing required packages...")
!{sys.executable} -m pip install -q "matplotlib" --no-cache-dir
!{sys.executable} -m pip install -q "torchvision" --no-cache-dir
!{sys.executable} -m pip install -q "torchmetrics" --no-cache-dir
!{sys.executable} -m pip install -q "ultralytics" --no-cache-dir

# OpenCV must be uninstalled and reinstalled last due to other packages installing OpenCV
print("Doing a fresh install of opencv-python-headless...")
!{sys.executable} -m pip uninstall -qy "opencv-python" "opencv-python-headless"
!{sys.executable} -m pip install -q "opencv-python-headless" --no-cache-dir

In [ ]:
import os
import urllib.request
from collections.abc import Sequence
from typing import Any, Callable

import numpy as np

# some initial imports
%matplotlib inline
%config InlineBackend.figure_format = "jpeg"  # Use JPEG format for inline visualizations

from matplotlib import pyplot as plt  # type: ignore
from PIL import Image

from nrtk.impls.perturb_image.generic.PIL.enhance import BrightnessPerturber

# Setup: Source image

In the next cell, we'll download and display a source image from the __[VisDrone](https://github.com/VisDrone/VisDrone-Dataset)__ dataset. The image will be cached in a local `data` subdirectory.

### A note on image storage

Typically in ML workflows, batches of images are processed as tensors of the color channels. Both our perturber (NRTK) and object detector (YOLO) accept numpy `ndarray` objects, and we will use [matplotlib.imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) to view them. The complication is that although YOLO inferences on `ndarray`, [it expects the color channels to be in BGR](https://docs.ultralytics.com/modes/predict) order. If we naively view the same data YOLO inferences on, the colors will be wrong; if we naively inference on what we view, the detections will be wrong. (Our NRTK perturbation is agnostic to the channel order.)

In this notebook, we'll convert the channel order to BGR when we load, and convert back whenever we explicitly call `imshow`.


In [ ]:
import nrtk

print(nrtk.__version__)
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)
img_path = os.path.join(data_dir, "visdrone_img.jpg")
if not os.path.isfile(img_path):
    url = "https://data.kitware.com/api/v1/item/623880f14acac99f429fe3ca/download"
    _ = urllib.request.urlretrieve(url, img_path)  # noqa: S310

img_pil = Image.open(img_path)
img_nd_bgr = np.asarray(img_pil)[
    :,
    :,
    ::-1,
]  # tip o' the hat to https://stackoverflow.com/questions/4661557/pil-rotate-image-colors-bgr-rgb
plt.figure()
plt.axis("off")

_ = plt.imshow(img_nd_bgr[:, :, ::-1])  # explicitly changing BGR to RGB for imshow

# NRTK Brightness perturbation: examples and guidance

The Brightness perturbation is set by a single floating point value `f`:

- `f == 0.0` (the minimum value): returns a black image.
- `0.0 < f < 1.0`: returns an image dimmer than the original.
- `f == 1.0`: returns the original image unchanged.
- `f > 1.0`: returns an image brighter than the original. There is no upper bound, but values greater than 2 or 3 start to wash out objects in a typical image, as seen below.
  

In [ ]:
_, ax = plt.subplots(2, 4, figsize=(10, 4))
for idx, f in enumerate((0.0, 0.25, 0.6, 1.0, 1.5, 2, 3, 6)):
    (row, col) = (int(idx / 4), idx % 4)
    bp = BrightnessPerturber(factor=f)
    ax[row, col].set_title(f"factor: {f}")
    ax[row, col].imshow(bp(img_nd_bgr)[0][:, :, ::-1])
    _ = ax[row, col].axis("off")
plt.tight_layout()

# Baseline detections

In the next cell, we'll download a [YOLOv11](https://docs.ultralytics.com/models/yolo11/) model, compute object detections on the source image, and display the results. As discussed above, these detections will serve as the "ground truth" for our relative mAP evaluation later.

*Note that here, we're using YOLO's built-in visualization tool, which automatically adjusts for BGR / RGB order.*

In [ ]:
# Import YOLO support
import ultralytics

ultralytics.checks()
print("Downloading model...")
model = ultralytics.YOLO("yolo11n.pt")
print("Computing baseline...")
baseline = model(img_nd_bgr)
baseline[0].show()

# MAITE Evaluation workflow preparation

We'll use the [MAITE Evaluation workflow](https://jatic.pages.jatic.net/cdao/maite/generated/maite.workflows.evaluate.html) to evaluate the performance of the perturbed data against our baseline detections. We'll need to "wrap" our model, data, and perturbations into callable objects to pass to the `maite.workflows.evaluate` function:

- We'll wrap the **model** to make predictions on input data when called.

- The wrapped **dataset** will return our test image when called. Note that this will be the original, unperturbed image; we'll apply our perturbations via...

- ...the **augmentation** object, which applies the perturbation to the image inside the evaluation.

- Finally, the **metric** object will define our precise scoring methodology.

The evaluation workflow in this notebook is slightly unusual. Typical ML workflows apply many different augmentations / perturbations to much larger datasets, and only call `evaluate` once to get a statistical view of performance. But since the goal of this notebook is to drill down into how perturbation affects performance, we've essentially flipped process, calling `evaluate` (and thus our wrapped objects) many times, once per loop on our single image perturbed to a known degree, and then observing how the metrics respond.

## Some helper classes

The following cell adds two classes to allow us to use YOLO detections with the MAITE evaluation workflow:

1. The `YOLODetectionTarget` helper class that stores the bounding boxes, label indices, and confidence scores for a single image's detections.

2. The `MaiteYOLODetection` adapter class that conforms to the MAITE [Object Detection Dataset](https://jatic.pages.jatic.net/cdao/maite/generated/maite.protocols.object_detection.Dataset.html) protocol by providing the `__len__` and `__getitem__` methods. The returned item is a tuple of (image, `YOLODetectionTarget`, metadata-dictionary).

In [ ]:
from dataclasses import dataclass

import torch

from nrtk.interop.maite.interop.object_detection.dataset import JATICObjectDetectionDataset

##
## Helper class for containing the boxes, label indices, and confidence scores.
##


@dataclass
class YOLODetectionTarget:
    """
    A helper class to represent object detection results in the format expected by YOLO-based models.

    Attributes:
        boxes (torch.Tensor): A tensor containing the bounding boxes for detected objects in
            [x_min, y_min, x_max, y_max] format.
        labels (torch.Tensor): A tensor containing the class labels for the detected objects.
            These may be floats for compatibility with specific datasets or tools.
        scores (torch.Tensor): A tensor containing the confidence scores for the detected objects.
    """

    boxes: torch.Tensor
    labels: torch.Tensor
    scores: torch.Tensor


##
## Prepare results for ingestion into maite dataset by puttin them into detection object
## Images must be channel first (c, h, w) in maite dataset objects
##
imgs = [np.transpose(img_nd_bgr, (2, 0, 1))]
dets = []
metadata = [{"id": 0}]
for _detection in baseline:
    boxes = baseline[0].boxes.xyxy.cpu()
    labels = baseline[0].boxes.cls.cpu()  # note, these are floats, not ints
    scores = baseline[0].boxes.conf.cpu()

    dets.append(YOLODetectionTarget(boxes, labels, scores))

## (1) Wrapping the detection model

The first object we'll wrap will be the detection model. The cell below defines a class adapting YOLO for the [MAITE Object Detection Model](https://jatic.pages.jatic.net/cdao/maite/generated/maite.protocols.object_detection.Model.html) protocol. The `__call__` method runs the model on images in the batch and is called by the MAITE evaluation workflow later in the notebook.

In [ ]:
import maite.protocols.object_detection as od
from maite.protocols import ArrayLike


class MaiteYOLODetector:
    """
    A wrapper class for a YOLO model to simplify its usage with input batches and object detection targets.

    This class takes a YOLO model instance, processes input image batches, and converts predictions into
    `YOLODetectionTarget` instances.

    Attributes:
        _model (ultralytics.models.yolo.model.YOLO): The YOLO model instance used for predictions.

    Methods:
        __call__(batch):
            Processes a batch of images through the YOLO model and returns the predictions as
            `YOLODetectionTarget` instances.
    """

    def __init__(self, model: ultralytics.models.yolo.model.YOLO) -> None:
        """
        Initializes the MaiteYOLODetector with a YOLO model instance.

        Args:
            model (ultralytics.models.yolo.model.YOLO): The YOLO model to use for predictions.
        """
        self._model = model

    def __call__(self, batch: Sequence[ArrayLike]) -> Sequence[YOLODetectionTarget]:
        """
        Processes a batch of images using the YOLO model and converts the predictions to `YOLODetectionTarget`
        instances.

        Args:
            batch (Sequence[ArrayLike]): A batch of images in (c, h, w) format (channel-first).

        Returns:
            Sequence[YOLODetectionTarget]: A list of YOLODetectionTarget instances containing the predictions for each
            image in the batch.
        """
        for i in range(len(batch)):
            # Convert images to channel-last format (h, w, c) for YOLO model
            batch[i] = np.transpose(batch[i], (1, 2, 0))

        yolo_predictions = self._model(batch, verbose=False)
        return [
            YOLODetectionTarget(
                p.boxes.xyxy.cpu(),  # Bounding boxes in (x_min, y_min, x_max, y_max) format
                p.boxes.cls.cpu(),  # Class indices for the detected objects
                p.boxes.conf.cpu(),  # Confidence scores for the detections
            )
            for p in yolo_predictions
        ]


# create the wrapped model object
yolo_model: od.Model = MaiteYOLODetector(model)

## (2) Wrapping the dataset

MAITE pairs images and their reference detections (aka targets, ground truth) into **datasets**. Typical ML workflows have many images per dataset; when these do not all fit in memory simultaneously, a *dataloader* object is used which can page images and annotations in from disk. For this notebook, however, each invocation of `evaluate` will use the same single-image dataset (our reference image with its baseline detections.)

In [ ]:
# our single image, its baseline detections, and a metadata dictionary
# switch image to channel first
single_image_dataset: od.Dataset = JATICObjectDetectionDataset(imgs, dets, metadata, dataset_id="visdrone_ex")

## (3) Wrapping the perturbations as augmentations

The `evaluate` function will perturb the image from the dataset using instances of the class defined below, one instance per perturbation value. Note that the object doesn't perform any augmentations until called by the `evaluate` workflow.

In [ ]:
from nrtk.interop.maite.interop.object_detection.augmentation import JATICDetectionAugmentation

bp = BrightnessPerturber(factor=1.0)
identity_augmentation = JATICDetectionAugmentation(bp, augment_id="identity")

## (4) Wrapping the metrics

We'll compare the detections in each perturbed image to the unperturbed detections using the Mean Average Precision (mAP) metric from the `torchmetrics` package. The following cell creates a mAP metrics object, wraps it in a MAITE [MAITE Object Detection Metric](https://jatic.pages.jatic.net/cdao/maite/generated/maite.protocols.object_detection.Metric.html) protocol-compatible class, and then creates an instance of this class, which will be called by `evaluate`.

This code is copied directly from the [MAITE object detection tutorial](https://jatic.pages.jatic.net/cdao/maite/tutorials/torchvision_object_detection.html#metrics) (with the exception of setting `class_metrics=True`.)

In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision

##
## Create an instance of the MAP metric object
##

tm_metric = MeanAveragePrecision(
    box_format="xyxy",
    iou_type="bbox",
    iou_thresholds=[0.5],
    rec_thresholds=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    max_detection_thresholds=[1, 10, 100],
    class_metrics=True,
    extended_summary=False,
    average="macro",
)

##
## This wrapper associates the MAP metric object with methods called by the evaluate
## workflow to accumulate detection data and compute the metrics.
##


class WrappedTorchmetricsMetric:
    """
    A wrapper class for a Torchmetrics metric designed to simplify its usage for object detection tasks.

    This class facilitates the conversion of object detection targets and predictions into the format
    expected by Torchmetrics metrics, allowing for easier integration with existing pipelines.

    Attributes:
        _tm_metric (Callable): The Torchmetrics metric to be wrapped, which takes lists of dictionaries
            containing torch.Tensor objects representing predictions and targets.

    Methods:
        to_tensor_dict(target):
            Converts an `ObjectDetectionTarget` into a dictionary format compatible with the Torchmetrics
            metric's `update` method.

        update(preds, targets):
            Updates the wrapped Torchmetrics metric with batches of predictions and targets in their native format.

        compute():
            Computes the final metric values using the wrapped Torchmetrics metric.

        reset():
            Resets the state of the wrapped Torchmetrics metric.
    """

    def __init__(
        self,
        tm_metric: Callable[[list[dict[str, torch.Tensor]], list[dict[str, torch.Tensor]]], dict[str, Any]],
    ) -> None:
        """
        Initializes the WrappedTorchmetricsMetric with the given Torchmetrics metric.

        Args:
            tm_metric (Callable): A Torchmetrics metric instance that expects predictions and targets as lists of
                dictionaries containing torch.Tensor objects.
        """
        self._tm_metric = tm_metric

    @staticmethod
    def to_tensor_dict(target: od.ObjectDetectionTarget) -> dict[str, torch.Tensor]:
        """
        Converts an ObjectDetectionTarget into a dictionary format compatible with the Torchmetrics metric's
        `update` method.

        Args:
            target (od.ObjectDetectionTarget): An object detection target instance containing boxes, labels, and scores.

        Returns:
            dict[str, torch.Tensor]: A dictionary with keys `boxes`, `scores`, and `labels`, each mapping to a tensor.
        """
        return {
            "boxes": torch.as_tensor(target.boxes),
            "scores": torch.as_tensor(target.scores),
            "labels": torch.as_tensor(target.labels).type(torch.int64),
        }

    def update(self, preds: od.TargetBatchType, targets: od.TargetBatchType) -> None:
        """
        Updates the wrapped Torchmetrics metric with the given predictions and targets.

        Args:
            preds (od.TargetBatchType): A batch of predictions in the format expected by the Torchmetrics metric.
            targets (od.TargetBatchType): A batch of targets in the format expected by the Torchmetrics metric.
        """
        preds_tm = [self.to_tensor_dict(pred) for pred in preds]
        targets_tm = [self.to_tensor_dict(tgt) for tgt in targets]
        self._tm_metric.update(preds_tm, targets_tm)

    def compute(self) -> dict[str, Any]:
        """
        Computes and returns the final metric values using the wrapped Torchmetrics metric.

        Returns:
            dict[str, Any]: A dictionary containing the computed metric values.
        """
        return self._tm_metric.compute()

    def reset(self) -> None:
        """Resets the state of the wrapped Torchmetrics metric, clearing any accumulated data."""
        self._tm_metric.reset()


##
## This is our instance variable that can compute the MAP metrics.
##

mAP_metric: od.Metric = WrappedTorchmetricsMetric(tm_metric)  # noqa: N816

# Running the evaluation

We now have all the wrappings required to evaluate our range of perturbations:
- The `yolo_model` object, wrapping the YOLO model
- The `single_image_dataset` object, providing our source image and its baseline detections
- The `augmentation` object, which when instantiated, applies a single perturbation value to its input
- The `mAP_metrics` object, defining the metrics to compute at each perturbation value

## Evaluation sanity check: ground truth against itself

Here we quickly check the evaluation workflow by creating an *identity augmentation* (with a brightness perturbation factor of 1.0, leaving the image unchanged) and scoring it.  The detections should also be unchanged from the baseline and thus give an mAP of 1.0.

In [ ]:
from maite.workflows import evaluate

# call the model for each image in the dataset (in this case, just the source image),
# scoring the resulting detections against those from the dataset
sanity_check_results, _, _ = evaluate(
    model=yolo_model,
    dataset=single_image_dataset,
    augmentation=identity_augmentation,
    metric=mAP_metric,
)

print("Sanity check: overall mAP (should be 1.0):", sanity_check_results["map"].item())

## Preparing the data

Now we'll prepare the augmentation instances for the evaluation. In the cell below, you can set three parameters for sweeping the set of perturbation values:
- **sweep_low**: the minimum perturbation value (must be >= 0)
- **sweep_high**: the maximum perturbation value
- **sweep_count**: how many perturbations to generation

You can also optionally select perturbations to visualize:
- **visualization_indices**: a list of perturbation indices *p*, 0 <= *p* < sweep_count. These instances will be rendered along with their corresponding detections.

In [ ]:
SWEEP_LOW = 0.2
SWEEP_HIGH = 2.0
SWEEP_COUNT = 30
VISUALIZATION_INDICES = [3, 9, 21]

##
## end user-settable parameters
##

perturbation_values = np.linspace(SWEEP_LOW, SWEEP_HIGH, SWEEP_COUNT, endpoint=True)
augmentations = [
    JATICDetectionAugmentation(BrightnessPerturber(p), augment_id=idx) for idx, p in enumerate(perturbation_values)
]

print(f"Generated {len(augmentations)} perturbation augmentations")

## Calling evaluate on the augmented data

We loop over all the augmentations, calling `evaluate` on each one and building up a list of resulting metrics for analysis.

Any augmentation indices specified above will be rendered in this step.

In [ ]:
perturbed_metrics = list()
for idx, a in enumerate(augmentations):
    # reset the metric object for each dataset
    mAP_metric.reset()
    result, _, _ = evaluate(model=yolo_model, dataset=single_image_dataset, augmentation=a, metric=mAP_metric)
    perturbed_metrics.append(result)

    if idx in VISUALIZATION_INDICES:
        # quickest way is to re-evaluate
        print(f"Perturbation #{idx}: brightness perturbation value {a.augment.factor:0.5}")
        datum = single_image_dataset[0]
        batch = ([datum[0]], [datum[1]], [datum[2]])
        # Extract the image from the augmentation and switch it to channel last
        aug = np.transpose(a(batch)[0][0], (1, 2, 0))
        _ = model(aug)[0].show()

# Evaluation analysis

Now we can plot how the metrics (for example, mAP @ IoU=50) vary with perturbation level, keeping in mind this is a **relative** mAP against the detections in the unperturbed image.


In [ ]:
map50_list = [m["map_50"].item() for m in perturbed_metrics]
plt.title("relative mAP@50")
plt.xlabel("Brightness perturbation factor")
plt.ylabel("relative mAP @ 50% IoU")
_ = plt.plot(perturbation_values, map50_list)

## Evaluation interpretation

Note that as plotted, the minimum y-axis value is 0.4. The metric shown, mAP@50, is the average precision of detections across all classes when the bounding box IoU is at least 0.5 (for more details, [see here](https://lightning.ai/docs/torchmetrics/stable/detection/mean_average_precision.html).) In general, we observe a perturbation value range between around 0.6 and 1.6 where the score is about 0.95 or higher, and sharp falloffs when the perturbation factor is below roughly 0.4 or above 1.75.  (Note the relative mAP is guaranteed to be 1.0 when the perturbation is 1.0, i.e. when the image is unchanged, the two detection sets are identical.)

## Additional plots

For further insight, we can plot the mAP per class:

In [ ]:
#
# Each instance of the metrics object has, potentially, a different set of observed classes.
# Loop through them to accumulate a unified set of classes to ensure consistent plotting across
# all thresholds.
#

unified_classes = set()
for m in perturbed_metrics:
    for class_idx in m["classes"].tolist():
        unified_classes.add(class_idx)

#
# dictionary of class_idx -> list of per-class mAP, or 0 if not present at that threshold
#

class_mAP = {class_idx: list() for class_idx in unified_classes}  # noqa: N816

#
# populate the lists across the perturbation values
#

for m in perturbed_metrics:
    this_perturbation_classes = m["classes"].tolist()
    for class_idx in unified_classes:
        if class_idx in this_perturbation_classes:
            # the index of the class in this individual metric instance
            this_class_idx = this_perturbation_classes.index(class_idx)
            class_mAP[class_idx].append(m["map_per_class"][this_class_idx].item())
        else:
            class_mAP[class_idx].append(0)

#
# plot
#

plt.title("Relative mAP per class")
plt.xlabel("Brightness perturbation factor")
plt.ylabel("relative mAP @ 50% IoU")
for class_idx, class_mAP_list in class_mAP.items():  # noqa: N816
    plt.plot(perturbation_values, class_mAP_list, label=baseline[0].names[class_idx])
plt.legend()
plt.show()

This plot shows several interesting results:

- The only "false alarm" **class** (with a mAP value of -1) is the appearance of one or more bicycles around 1.24. There are no bicycles in the "ground truth" unperturbed detections (but again, since this a relative mAP, this does not necessarily mean there are no true bicycles in the image.)

- The truck, person, and motorcycle classes are much more robust to brightness perturbations than the car class. Examination of the unperturbed and a sample low-illumination image suggest a few possibilities:

    - There are many more car instances (15) than non-car (5 persons, 1 motorcycle, 2 trucks) in the unperturbed image.
    - The non-car classes detected in the unperturbed image are all in the foreground, and are thus larger. People, in particular, are distributed throughout the image, but only detected when relatively close and large.

Any conclusions about classification accuracy should be considered in light of these caveats. In particular, the foreground positioning of the detected non-car objects suggests that instead of looking at per-class results, we drill down by bounding box area. Fortunately, the metrics class supports this:


In [ ]:
plt.title("relative mAP values per area")
plt.xlabel("Brightness perturbation factor")
plt.ylabel("relative mAP")
for k in ("map", "map_small", "map_medium"):
    plt.plot(perturbation_values, [m[k].item() for m in perturbed_metrics], label=k)
plt.legend()
plt.show()

The `map` line covers all sizes; `map_small` and `map_medium` are the mean average precision for objects (smaller than 32^2 pixels, between 32^2 and 96^2 pixels) in area, respectively. (There are no detections in the `map_large` category.) (Here, the mAP value is averaged over a **range** of IoU thresholds, between 0.5 and 0.95.) We see that medium objects, regardless of class, are generally much more robust to brighter illumination perturbations than small ones, which makes sense when observing how objects tend to get "washed out" in the perturbation examples shown in the [Examples and Guidance](#NRTK-Brightness-perturbation:-examples-and-guidance) section above. For dimming perturbations, the situation is reversed (although not as dramatically): small objects tend to be more robust than medium, but both fall off dramatically around 0.3.


# End of notebook